In [1]:
%matplotlib inline

In [1]:
import py4j
py4j.__version__

'0.10.8.1'

In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e
from py4j.java_gateway import get_field
ok, r=track(lambda a: s().testScv(defaultValue=5.5, message="hello world"))
print(ok, r) 

✔ testScv default 21 2019-02-16 17:03:08.087 ➷ 14 ms
True {'responseMessage': 'success', 'resp': 'service done'}


In [11]:
from sagas.ofbiz.entities import OfEntity as e, finder
from py4j.protocol import Py4JError

entity="Tests"
limit=1
start=5

try:
    result = finder.find_list(entity, limit, start)
    print(oc.j.ValueHelper.valueListToJson(result))
except Py4JError as e:
    print(e.errmsg+e.java_exception.getMessage())    

An error occurred while calling o141.findList.
org.apache.ofbiz.entity.GenericModelException: Could not find definition for entity name Tests (Could not find definition for entity name Tests)


In [2]:
from sagas.ofbiz.services import OfService as s, oc, track
ok, r=await s('srv').testScv(defaultValue='x', message="hello world")
print(ok,r)

False {'_result': 1, 'messages': ['Type conversion of field [defaultValue] to type [Double] failed for value "x": org.apache.ofbiz.base.util.GeneralException: java.text.ParseException: Unparseable number: "x" (java.text.ParseException: Unparseable number: "x")']}


In [3]:
ok, r=await s('srv').testScv(defaultValue=6.6, message="hello world")
print(ok,r)

True {'responseMessage': 'success', 'resp': 'service done', '_result': 0}


In [2]:
import asyncio
import uuid
import json
from sagas.ofbiz.srv_client import SrvClient

loop = asyncio.get_event_loop()
rpc = await SrvClient(loop).connect()
print(" [x] Requesting ...")
json_pars = json.dumps({'_service': 'testScv',
                        'defaultValue': 5.5,
                        'message': "hello world"})
response = await rpc.call(json_pars)
print(" [.] Got %s" % response)

 [x] Requesting ...
 [.] Got {"responseMessage":"success","resp":"service done","_result":0}


In [3]:
json_pars = json.dumps({'_service': 'testScv',
                        'defaultValue': 5.5,
                        'message': "hello world"})
response = await rpc.call(json_pars)
print(" [.] Got %s" % response)

 [.] Got {"responseMessage":"success","resp":"service done","_result":0}


In [4]:
def params(**kwargs):
    print(json.dumps(kwargs))
params(defaultValue=5.5, message="hello world")

{"defaultValue": 5.5, "message": "hello world"}


In [3]:
import json
json_pars=json.dumps({'defaultValue':5.5, 'message':"hello world"})

srv=oc.j.ServiceInvoker(oc.dispatcher, oc.delegator, "testScv", json_pars)
ret=srv.invoke()
print(ret, ret.getID())
json_r=srv.getJsonResult()
print(json.dumps(json.loads(json_r), indent=2))

Success 0
{
  "responseMessage": "success",
  "resp": "service done"
}


In [13]:
result=srv.getResult()
json_r=oc.j.ValueHelper.mapToJson(result)
# print(json_r)
print(json.dumps(json.loads(json_r), indent=2))

{
  "responseMessage": "success",
  "resp": "service done"
}


In [6]:
async def multiply(*, x, y):
    return x * y

In [12]:
r=await multiply(x=5,y=5)
print(r)

25


In [14]:
# import pandas as pd
# df=pd.read_json(json_r, orient='index')
# df

In [15]:
from IPython.display import display, JSON
JSON(json.loads(json_r))

<IPython.core.display.JSON object>

In [27]:
def service_gen(name):
    serv_model=oc.service_model(name)
    params=serv_model.getModelParamList()
    for param in params:
        internal = get_field(param, "internal")
        p_name = get_field(param, "name")
        p_type=get_field(param, 'type')
        p_mode=get_field(param, 'mode')
        # print(p_name, internal)
        if internal:
            pass
        else:
            print(p_mode, p_name, p_type)

service_gen('testScv')

IN defaultValue Double
IN message String
OUT resp String


In [4]:
from sagas.ofbiz.service_gen import get_all_regular_services
services=get_all_regular_services()
print(len(services))

service addAdditionalViewForProduct parameter uploadedFile's type is ByteBuffer
service addImageForProductPromo parameter uploadedFile's type is ByteBuffer
service addMultipleuploadForProduct parameter uploadedFile's type is ByteBuffer
service attachUploadToDataResource parameter uploadedFile's type is ByteBuffer
service buildSurveyFromPdf parameter inputByteBuffer's type is ByteBuffer
service buildSurveyResponseFromPdf parameter inputByteBuffer's type is ByteBuffer
service byteBufferTest parameter byteBuffer1's type is ByteBuffer
service createAnonFile parameter binData's type is ByteBuffer
service createArticleContent parameter uploadedFile's type is ByteBuffer
service createBlogEntry parameter uploadedFile's type is ByteBuffer
service createBulkProductPromoCode parameter uploadedFile's type is ByteBuffer
service createBulkProductPromoCodeEmail parameter uploadedFile's type is ByteBuffer
service createCatalinaSession parameter sessionInfo's type is ByteBuffer
service createCommConten

In [37]:
from ruamel.yaml import YAML
yaml = YAML()
with open('.services.yml', 'w') as outfile:
    yaml.dump(services, outfile)

In [40]:
with open('.services.yml') as fp:
    str_data = fp.read()
data = yaml.load(str_data)
print(len(data), type(data))
print(data[:5])

3345 <class 'ruamel.yaml.comments.CommentedSeq'>
['AddProductToBestSellCategory', 'FindBestSellingProduct', 'FindCategoryChild', 'MoveItemBetweenShipGroups', 'RemoveProductFromBestSellCategory']


In [26]:
print('hel'.find('x'))
print('oF'.lower())

-1
of


In [27]:
def norm_loc(loc):
    pkg_prefixes=['ofbiz-framework/applications', 
                  'ofbiz-framework/framework',
                  'ofbiz-framework/plugins']
    for pkg_prefix in pkg_prefixes:
        idx=loc.find(pkg_prefix)
        if idx!=-1:
            return loc[idx+len(pkg_prefix)+1:].replace('servicedef/','').replace('.xml','').replace('/','_').lower()
    raise ValueError('Cannot normalize the location '+loc)
    
def get_service_package(srv):
    serv_model = oc.service_model(srv)
    def_loc = get_field(serv_model, 'definitionLocation')
    return norm_loc(def_loc)

In [28]:
# from sagas.ofbiz.service_gen import get_service_package
service_groups={}
for srv in services:
    pkg=get_service_package(srv)
    if pkg in service_groups:
        grp=service_groups[pkg]
        grp.append(srv)
    else:
        service_groups[pkg]=[srv]

print(len(service_groups))

111


In [29]:
for k in service_groups.keys():
    print(k)

product_services
order_services
accounting_services_sagepay
common_services
party_services
accounting_services_perm
accounting_services_valuelink
product_services_facility
order_services_shoppinglist
accounting_services_fixedasset
accounting_services_finaccount
accounting_services_admin
product_services_shipment
marketing_services
manufacturing_services_bom
manufacturing_services_production_run
order_services_return
common_services_security
order_services_requirement
workeffort_services_timesheet
content_services_website
accounting_services_invoice
accounting_services_authorizedotnet
accounting_services_paymentmethod
product_services_feature
humanres_services
workeffort_services
content_services_content
product_services_maint
service_services
order_services_quote
common_services_test
content_services_survey
accounting_services_billing
accounting_services_tax
accounting_services_ledger
product_services_cost
product_services_pricepromo
accounting_services_payment
product_services_picklis

In [41]:
srvs=service_groups['humanres_services_position']
print(len(srvs))

6


In [12]:
from sagas.ofbiz.entities import OfEntity as e
from sagas.ofbiz.entity_gen import norm_package

def get_entity_package_def(entity_name):
    model = oc.delegator.getModelEntity(entity_name)
    return norm_package(model.getPackageName())

print(get_entity_package_def('Person'))

party_party


In [42]:
def proc_service_refs(serv_name, deps):
    model_serv = oc.service_model(serv_name)
    def_ent = get_field(model_serv, 'defaultEntityName')
    if def_ent != "":
        ent_pkg=get_entity_package_def(def_ent)
        deps.add(ent_pkg)

serv_grp='humanres_services_position'
srvs=service_groups[serv_grp]
deps=set()
for srv in srvs:
    proc_service_refs(srv, deps)

print(deps)

{'humanres_position'}
